# Test Durbina-Watsona - Prosty Przykład

Ten notebook pokazuje praktyczne przykłady użycia testu Durbina-Watsona do wykrywania autokorelacji w resztach modelu regresji.

## 1. Import bibliotek

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from statsmodels.stats.stattools import durbin_watson
from statsmodels.graphics.tsaplots import plot_acf

# Ustawienia
np.random.seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("Biblioteki załadowane pomyślnie!")

## 2. Przykład 1: Model BEZ autokorelacji (prawidłowy)

Zbudujemy prosty model liniowy z resztami niezależnymi od siebie.

In [ ]:
# Generowanie danych
n = 100
X = np.linspace(0, 10, n).reshape(-1, 1)

# Prawdziwa zależność: y = 2 + 3*x + szum losowy
y_true = 2 + 3 * X.flatten()
noise = np.random.normal(0, 1, n)  # Szum NIEZALEŻNY
y = y_true + noise

# Regresja liniowa
model = LinearRegression()
model.fit(X, y)
y_pred = model.predict(X)
residuals = y - y_pred

print(f"Współczynniki modelu:")
print(f"  a (wyraz wolny) = {model.intercept_:.4f}")
print(f"  b (nachylenie)  = {model.coef_[0]:.4f}")
print(f"\nPrawdziwe wartości: a = 2, b = 3")

### Test Durbina-Watsona

In [ ]:
# Obliczanie statystyki Durbina-Watsona
dw_stat = durbin_watson(residuals)

print("=" * 60)
print("TEST DURBINA-WATSONA - PRZYKŁAD 1")
print("=" * 60)
print(f"\nStatystyka DW: {dw_stat:.4f}")
print(f"\nInterpretacja:")

if 1.5 < dw_stat < 2.5:
    print("✓ BRAK autokorelacji (DW ≈ 2.0)")
    print("  Model jest poprawnie specyfikowany")
    print("  Reszty są niezależne od siebie")
elif dw_stat <= 1.5:
    print("✗ DODATNIA autokorelacja (DW < 1.5)")
    print("  Problem: kolejne reszty są podobne do siebie")
else:
    print("✗ UJEMNA autokorelacja (DW > 2.5)")
    print("  Problem: kolejne reszty zmieniają znak")

# Przybliżony współczynnik autokorelacji
rho_approx = 1 - dw_stat / 2
print(f"\nPrzybliżony współczynnik autokorelacji ρ: {rho_approx:.4f}")

### Wizualizacja

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Dopasowanie modelu
axes[0, 0].scatter(X, y, alpha=0.6, label='Dane obserwowane', s=50)
axes[0, 0].plot(X, y_pred, 'r-', linewidth=2, label='Linia regresji')
axes[0, 0].set_xlabel('X', fontsize=12)
axes[0, 0].set_ylabel('Y', fontsize=12)
axes[0, 0].set_title('Model regresji liniowej', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Reszty w czasie
axes[0, 1].plot(residuals, marker='o', linestyle='-', alpha=0.6, markersize=4)
axes[0, 1].axhline(y=0, color='red', linestyle='--', linewidth=2, label='y=0')
axes[0, 1].set_xlabel('Obserwacja', fontsize=12)
axes[0, 1].set_ylabel('Reszta', fontsize=12)
axes[0, 1].set_title(f'Reszty w czasie (DW = {dw_stat:.4f})', fontsize=14, fontweight='bold')
axes[0, 1].legend(fontsize=10)
axes[0, 1].grid(True, alpha=0.3)

# 3. Lag plot - reszty vs reszty opóźnione
axes[1, 0].scatter(residuals[:-1], residuals[1:], alpha=0.6, s=50)
axes[1, 0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].axvline(x=0, color='red', linestyle='--', alpha=0.5)
axes[1, 0].set_xlabel('Reszta(t)', fontsize=12)
axes[1, 0].set_ylabel('Reszta(t+1)', fontsize=12)
axes[1, 0].set_title('Lag Plot - sprawdzenie autokorelacji', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

# 4. Funkcja autokorelacji (ACF)
plot_acf(residuals, lags=20, ax=axes[1, 1], alpha=0.05)
axes[1, 1].set_title('Funkcja Autokorelacji (ACF)', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Opóźnienie', fontsize=12)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETACJA WYKRESÓW:")
print("  • Lag plot: Brak wyraźnego wzorca = brak autokorelacji")
print("  • ACF: Wszystkie opóźnienia w granicach przedziału ufności = OK")

## 3. Przykład 2: Model Z dodatnią autokorelacją (problem!)

Teraz stworzymy model, w którym reszty są skorelowane ze sobą.

In [ ]:
# Generowanie danych z autokorelowanym szumem
X2 = np.linspace(0, 10, n).reshape(-1, 1)
y2_true = 2 + 3 * X2.flatten()

# Szum z autokorelacją AR(1): e_t = 0.7 * e_{t-1} + ν_t
rho = 0.7  # Współczynnik autokorelacji
noise_ar = np.zeros(n)
noise_ar[0] = np.random.normal(0, 1)

for t in range(1, n):
    noise_ar[t] = rho * noise_ar[t-1] + np.random.normal(0, 1)

y2 = y2_true + noise_ar

# Regresja
model2 = LinearRegression()
model2.fit(X2, y2)
y2_pred = model2.predict(X2)
residuals2 = y2 - y2_pred

print(f"Współczynniki modelu 2:")
print(f"  a (wyraz wolny) = {model2.intercept_:.4f}")
print(f"  b (nachylenie)  = {model2.coef_[0]:.4f}")
print(f"\nWprowadzona autokorelacja: ρ = {rho}")

### Test Durbina-Watsona dla modelu z autokorelacją

In [ ]:
# Test DW
dw_stat2 = durbin_watson(residuals2)

print("=" * 60)
print("TEST DURBINA-WATSONA - PRZYKŁAD 2 (z autokorelacją)")
print("=" * 60)
print(f"\nStatystyka DW: {dw_stat2:.4f}")
print(f"\nInterpretacja:")

if 1.5 < dw_stat2 < 2.5:
    print("✓ BRAK autokorelacji (DW ≈ 2.0)")
elif dw_stat2 <= 1.5:
    print("✗ DODATNIA autokorelacja (DW < 1.5)")
    print("  ⚠ PROBLEM: Reszty są skorelowane!")
    print("\n  Możliwe rozwiązania:")
    print("    1. Dodaj opóźnioną zmienną zależną Y_{t-1}")
    print("    2. Dodaj brakujące zmienne objaśniające")
    print("    3. Użyj modeli szeregów czasowych (ARIMA)")
    print("    4. Zastosuj estymację z korektą (Newey-West)")
else:
    print("✗ UJEMNA autokorelacja (DW > 2.5)")

# Przybliżony współczynnik autokorelacji
rho_approx2 = 1 - dw_stat2 / 2
print(f"\nPrzybliżony współczynnik autokorelacji ρ: {rho_approx2:.4f}")
print(f"Prawdziwy współczynnik autokorelacji:     {rho:.4f}")
print(f"\nTeoretyczna wartość DW ≈ 2(1-ρ) = {2*(1-rho):.4f}")

### Wizualizacja modelu z autokorelacją

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# 1. Dopasowanie modelu
axes[0, 0].scatter(X2, y2, alpha=0.6, label='Dane obserwowane', s=50, color='orange')
axes[0, 0].plot(X2, y2_pred, 'r-', linewidth=2, label='Linia regresji')
axes[0, 0].set_xlabel('X', fontsize=12)
axes[0, 0].set_ylabel('Y', fontsize=12)
axes[0, 0].set_title('Model z autokorelowanymi resztami', fontsize=14, fontweight='bold')
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(True, alpha=0.3)

# 2. Reszty w czasie - widoczny wzorzec!
axes[0, 1].plot(residuals2, marker='o', linestyle='-', alpha=0.7, 
                markersize=4, color='red', linewidth=1.5)
axes[0, 1].axhline(y=0, color='black', linestyle='--', linewidth=2)
axes[0, 1].set_xlabel('Obserwacja', fontsize=12)
axes[0, 1].set_ylabel('Reszta', fontsize=12)
axes[0, 1].set_title(f'Reszty w czasie (DW = {dw_stat2:.4f}) - PROBLEM!', 
                     fontsize=14, fontweight='bold', color='red')
axes[0, 1].grid(True, alpha=0.3)

# 3. Lag plot - wyraźna dodatnia korelacja!
axes[1, 0].scatter(residuals2[:-1], residuals2[1:], alpha=0.6, s=50, color='red')
axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1, 0].axvline(x=0, color='black', linestyle='--', alpha=0.5)

# Dodaj linię trendu
z = np.polyfit(residuals2[:-1], residuals2[1:], 1)
p = np.poly1d(z)
x_line = np.linspace(residuals2[:-1].min(), residuals2[:-1].max(), 100)
axes[1, 0].plot(x_line, p(x_line), "b--", linewidth=2, label=f'Trend (nachylenie={z[0]:.3f})')

axes[1, 0].set_xlabel('Reszta(t)', fontsize=12)
axes[1, 0].set_ylabel('Reszta(t+1)', fontsize=12)
axes[1, 0].set_title('Lag Plot - Wyraźna dodatnia autokorelacja!', 
                     fontsize=14, fontweight='bold', color='red')
axes[1, 0].legend(fontsize=10)
axes[1, 0].grid(True, alpha=0.3)

# 4. ACF - istotne opóźnienia!
plot_acf(residuals2, lags=20, ax=axes[1, 1], alpha=0.05)
axes[1, 1].set_title('ACF - Wiele istotnych opóźnień!', 
                     fontsize=14, fontweight='bold', color='red')
axes[1, 1].set_xlabel('Opóźnienie', fontsize=12)

plt.tight_layout()
plt.show()

print("\n💡 INTERPRETACJA WYKRESÓW:")
print("  • Reszty w czasie: Widoczny wzorzec - kolejne wartości 'podążają' za sobą")
print("  • Lag plot: Dodatnie nachylenie = dodatnia autokorelacja")
print("  • ACF: Wiele opóźnień poza przedziałem ufności = autokorelacja")

## 4. Porównanie różnych poziomów autokorelacji

Zobaczmy jak zmienia się statystyka DW dla różnych poziomów autokorelacji.

In [ ]:
# Różne poziomy autokorelacji
rho_values = [0.0, 0.3, 0.5, 0.7, 0.9]
results = []

print("=" * 70)
print("PORÓWNANIE RÓŻNYCH POZIOMÓW AUTOKORELACJI")
print("=" * 70)
print(f"\n{'ρ (autok.)':<12} {'DW stat':<12} {'DW teoria':<12} {'Status':<20}")
print("-" * 70)

for rho in rho_values:
    # Generowanie danych
    X_temp = np.linspace(0, 10, n).reshape(-1, 1)
    y_temp_true = 2 + 3 * X_temp.flatten()
    
    # Szum z autokorelacją
    noise_temp = np.zeros(n)
    noise_temp[0] = np.random.normal(0, 1)
    for t in range(1, n):
        noise_temp[t] = rho * noise_temp[t-1] + np.random.normal(0, 1)
    
    y_temp = y_temp_true + noise_temp
    
    # Regresja
    model_temp = LinearRegression()
    model_temp.fit(X_temp, y_temp)
    y_temp_pred = model_temp.predict(X_temp)
    residuals_temp = y_temp - y_temp_pred
    
    # Test DW
    dw_temp = durbin_watson(residuals_temp)
    dw_theory = 2 * (1 - rho)  # Teoretyczna wartość
    
    # Status
    if 1.5 < dw_temp < 2.5:
        status = "OK ✓"
    elif dw_temp <= 1.5:
        status = "Problem (dodatnia) ✗"
    else:
        status = "Problem (ujemna) ✗"
    
    results.append({'rho': rho, 'DW': dw_temp, 'DW_theory': dw_theory, 'status': status})
    print(f"{rho:<12.1f} {dw_temp:<12.4f} {dw_theory:<12.4f} {status:<20}")

print("\n💡 WNIOSEK: Im wyższa autokorelacja (ρ), tym niższa statystyka DW")
print("           Formuła: DW ≈ 2(1 - ρ)")

### Wykres zależności DW od ρ

In [ ]:
# Wizualizacja zależności
rho_arr = np.array([r['rho'] for r in results])
dw_arr = np.array([r['DW'] for r in results])
dw_theory_arr = np.array([r['DW_theory'] for r in results])

plt.figure(figsize=(12, 6))
plt.plot(rho_arr, dw_arr, 'o-', linewidth=2, markersize=10, label='DW empiryczne', color='blue')
plt.plot(rho_arr, dw_theory_arr, 's--', linewidth=2, markersize=8, label='DW teoretyczne: 2(1-ρ)', color='red')

# Strefy
plt.axhspan(1.5, 2.5, alpha=0.2, color='green', label='Strefa OK')
plt.axhspan(0, 1.5, alpha=0.2, color='red', label='Dodatnia autokorelacja')
plt.axhspan(2.5, 4, alpha=0.2, color='orange', label='Ujemna autokorelacja')

plt.xlabel('Współczynnik autokorelacji (ρ)', fontsize=12)
plt.ylabel('Statystyka Durbina-Watsona', fontsize=12)
plt.title('Zależność DW od poziomu autokorelacji', fontsize=14, fontweight='bold')
plt.legend(fontsize=10, loc='best')
plt.grid(True, alpha=0.3)
plt.ylim(0, 4)
plt.tight_layout()
plt.show()

## 5. Funkcja pomocnicza do analizy DW

In [ ]:
def analyze_durbin_watson(residuals, model_name="Model"):
    """
    Funkcja do kompleksowej analizy testu Durbina-Watsona
    
    Parameters:
    -----------
    residuals : array-like
        Reszty z modelu regresji
    model_name : str
        Nazwa modelu do wyświetlenia
    """
    dw_stat = durbin_watson(residuals)
    rho_approx = 1 - dw_stat / 2
    
    print("=" * 60)
    print(f"ANALIZA DURBINA-WATSONA: {model_name}")
    print("=" * 60)
    print(f"\nStatystyka DW:        {dw_stat:.4f}")
    print(f"Przybliżone ρ:        {rho_approx:.4f}")
    print(f"Liczba obserwacji:    {len(residuals)}")
    
    print("\nInterpretacja:")
    if 1.5 < dw_stat < 2.5:
        print("  ✓ BRAK autokorelacji (OK)")
        print("  Model spełnia założenie o niezależności reszt")
    elif dw_stat <= 1.5:
        print("  ✗ DODATNIA autokorelacja (PROBLEM)")
        print("  Kolejne reszty są podobne do siebie")
        print("\n  Możliwe rozwiązania:")
        print("    • Dodaj opóźnioną zmienną zależną")
        print("    • Dodaj brakujące zmienne objaśniające")
        print("    • Sprawdź trend lub sezonowość")
        print("    • Użyj modeli szeregów czasowych")
    else:
        print("  ✗ UJEMNA autokorelacja (PROBLEM)")
        print("  Kolejne reszty zmieniają znak")
    
    print("\nGranice interpretacyjne:")
    print("  DW ≈ 2.0  : Brak autokorelacji (ideał)")
    print("  1.5 < DW < 2.5 : Akceptowalne")
    print("  DW < 1.5  : Dodatnia autokorelacja")
    print("  DW > 2.5  : Ujemna autokorelacja")
    print("=" * 60)
    
    return {'DW': dw_stat, 'rho': rho_approx}

# Test funkcji na naszych przykładach
print("\n\n")
result1 = analyze_durbin_watson(residuals, "Model bez autokorelacji")
print("\n\n")
result2 = analyze_durbin_watson(residuals2, "Model z autokorelacją")

## 6. Podsumowanie

### Czego się nauczyliśmy:

1. **Test Durbina-Watsona** wykrywa autokorelację pierwszego rzędu w resztach modelu regresji

2. **Interpretacja wartości DW:**
   - DW ≈ 2.0 → Brak autokorelacji (ideał)
   - DW < 1.5 → Dodatnia autokorelacja (problem)
   - DW > 2.5 → Ujemna autokorelacja (problem)

3. **Związek z współczynnikiem autokorelacji:**
   - DW ≈ 2(1 - ρ)
   - Im wyższa autokorelacja, tym niższa statystyka DW

4. **Diagnostyka wizualna:**
   - **Reszty w czasie** - losowy wzorzec = OK
   - **Lag plot** - brak trendu = OK
   - **ACF** - wszystkie w granicach = OK

5. **Konsekwencje autokorelacji:**
   - Niedoszacowane błędy standardowe
   - Niewiarygodne testy istotności
   - Nieefektywne prognozy

## 7. Zadania do samodzielnego eksperymentowania

Spróbuj:

1. Zmienić współczynnik autokorelacji `rho` i zobaczyć jak zmienia się DW
2. Stworzyć model z ujemną autokorelacją (ρ < 0)
3. Użyć rzeczywistych danych i przetestować autokorelację
4. Porównać test DW z testem Breuscha-Godfreya
5. Dodać opóźnioną zmienną zależną i zobaczyć czy rozwiązuje problem autokorelacji